In [ ]:
from openai import OpenAI
import os
from pathlib import Path

In [ ]:
# Load environment variables from .env file
client = OpenAI(
    base_url=os.getenv("OPENAI_API_URL"), api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
# [RUN ONCE] Uploads a file and creates a vector store and assistant
client = OpenAI(
    base_url=os.getenv("OPENAI_API_URL"), api_key=os.getenv("OPENAI_API_KEY")
)

# Upload a file and create a vector store
create_file_response = client.files.create(
    file=Path("../../data/jack_and_diane.md"),
    purpose="assistants",
)

file_id = create_file_response.model_dump().get("id")

create_vector_store_response = client.beta.vector_stores.create(
    file_ids=[file_id], name="jack_and_diane"
)

vector_store_id = create_vector_store_response.model_dump().get("id")

create_assistant_response = client.beta.assistants.create(
    name="jack_and_diane",
    model="gpt-4-turbo",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}},
)

assistant_id = create_assistant_response.model_dump().get("id")

print(
    f"Assistant ID: {assistant_id} \nVector Store ID: {vector_store_id} \nFile ID: {file_id}"
)

In [ ]:
# List files and assistants
list_assistants = client.beta.assistants.list().data

print(f"Assistants: {list_assistants}\n")

list_vector_stores = client.beta.vector_stores.list().data

print(f"Vector Stores: {list_vector_stores} \n")

list_files = client.files.list().data

print(f"Files: {list_files} \n")

In [ ]:
# Get the ID for the assistant 'jack_and_diane'
assistant_id = None

for assistant in list_assistants:
    if assistant.name == "jack_and_diane":
        assistant_id = assistant.id
        break

print(f"The ID for the assistant 'jack_and_diane' is: {assistant_id}")

In [ ]:
# Create a new assistant
thread = client.beta.threads.create()

In [ ]:
# Send a message to the assistant
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the name of Jack's cat?",
)

In [ ]:
# Start a run
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="You are an expert in the love story of Jack and Diane. Answer the user's question.",
)

In [ ]:
# If the run is completed, print the messages
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages)
else:
    print("The run was not completed.")
    print(run.status)

In [ ]:
# Create, Retrieve, Delete a file
create_response = client.files.create(
    file=Path("data/test_file.md"),
    purpose="assistants",
)

print(f"Created: {create_response}")

get_response = client.files.retrieve(
    file_id=create_response.model_dump()["id"],
)

print(f"Retrieved: {get_response}")

delete_response = client.files.delete(
    file_id=create_response.model_dump()["id"],
)

print(f"Deleted: {delete_response}")

In [ ]:
# Create, Retrieve, Delete an assistant
create_response = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "file_search"}],
    model="gpt-4-turbo",
)

print(f"Created: {create_response}")

get_response = client.beta.assistants.retrieve(
    assistant_id=create_response.model_dump()["id"],
)

print(f"Retrieved: {get_response}")


delete_response = client.beta.assistants.delete(
    assistant_id=create_response.model_dump()["id"],
)

print(f"Deleted: {delete_response}")